In [3]:
# Importing necesssary libraries
import pandas as pd
import re

# Read the dataset
df_orgs = pd.read_csv("orgs_full.csv")
df_exs  = pd.read_csv("exercises_full.csv")

# Info and data types of the datasets & columns
df_orgs.info(), df_exs.info(), df_ttps.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ORGID                40 non-null     int64 
 1   Industry             40 non-null     object
 2   Region               40 non-null     object
 3   Size                 40 non-null     object
 4   SecurityBudget       40 non-null     object
 5   PrimarySecurityTeam  40 non-null     object
 6   Maturity             40 non-null     int64 
 7   Complexity           40 non-null     int64 
 8   ExerciseFrequency    40 non-null     int64 
 9   Threats              40 non-null     object
 10  TTPs                 40 non-null     object
 11  Aims                 40 non-null     object
dtypes: int64(4), object(8)
memory usage: 3.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  -----

(None, None, None)

In [4]:
# Lambda: normalize one column name
norm_col = lambda c: str(c).strip().lower().replace(" ", "_")
def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.rename(columns=norm_col)

# Lets clean the text specially multivalued separated by ;
# splits → trims → removes blanks → sorts → rejoins
def normalize_list_col(s: pd.Series):
    return (s.fillna('')
             .str.split(';')
             .apply(lambda xs: ';'.join(sorted(x.strip() for x in xs if x.strip()))))

# Lets fix the case of the categorical variables
# Convert text to lower case.
def normalise_case(s: pd.Series):
    return (s.astype(str)
         .str.strip()
         .str.replace(r'\s+', ' ', regex=True)
         .str.lower())

In [5]:
# first of all lets normalise the columna names and make it consistent among all dataset.. best practices..
df_orgs = normalize_columns(df_orgs)
df_exs = normalize_columns(df_exs)


df_orgs.columns.values, df_exs.columns.values, df_ttps.columns.values

(array(['orgid', 'industry', 'region', 'size', 'securitybudget',
        'primarysecurityteam', 'maturity', 'complexity',
        'exercisefrequency', 'threats', 'ttps', 'aims'], dtype=object),
 array(['exid', 'excreation', 'exthreat', 'exttps', 'excategories',
        'exgroups', 'exsoftware', 'exstructure', 'exmaturity',
        'excomplexity', 'exlength', 'exaudience', 'extradecraftintra',
        'extradecraftinter'], dtype=object),
 array(['Name', ' Id'], dtype=object))

In [13]:
# ---------- helpers fucntion to clean dataset ----------

def norm_col(c: str) -> str:
    return str(c).strip().lower().replace(" ", "_")

def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    return df.rename(columns=norm_col)

def split_semicolons(val):
    if pd.isna(val) or not str(val).strip():
        return []
    return [p.strip() for p in str(val).split(";") if p.strip()]

def norm_token(s: str) -> str:
    s = str(s).strip().lower()
    s = " ".join(s.split())
    return s.replace(" ", "_")

def bucket_numeric(v, bins, labels):
    if pd.isna(v):
        return "unknown"
    idx = np.digitize([float(v)], bins=bins, right=True)[0]
    idx = min(idx, len(labels)-1)
    return labels[idx]


In [27]:
# Lets concentrate on the exercise dataset only for now..

# Expected exercise feature/columns.. after normalisation
EXPECTED_EX = [
    "exid","exthreat","exttps","excategories","exgroups","exsoftware",
    "exstructure","exaudience","exmaturity","excomplexity","exlength",
    "extradecraftintra","extradecraftinter"
]
missing = [c for c in EXPECTED_EX if c not in df_exs.columns]
if missing: print("misssing exercise cols:", missing)

def tokens_for_exercise(row):
    toks = []
    # Multi-valued categorical → prefixed tokens
    for field, prefix in [
        ("exthreat",    "threat_"),
        ("exttps",      "ttp_"),
        ("excategories","cat_"),
        ("exgroups",    "group_"),
        ("exsoftware",  "soft_"),
        ("exstructure", "struct_"),
        ("exaudience",  "aud_"),
    ]:
        for val in split_semicolons(row.get(field, "")):
            toks.append(prefix + norm_token(val))

    # Numeric → buckets → tokens
    mat = bucket_numeric(row.get("exmaturity", np.nan),   bins=[2,3,5],   labels=["L","M","H"])
    cpx = bucket_numeric(row.get("excomplexity", np.nan), bins=[2,3,5],   labels=["L","M","H"])
    dur = bucket_numeric(row.get("exlength", np.nan),     bins=[60,120,1e9], labels=["short","medium","long"])
    toks += [f"maturity_{mat}", f"complexity_{cpx}", f"len_{dur}"]

    # Tradecraft scores (0..1) → coarse buckets
    for field, prefix in [("extradecraftintra","td_intra_"), ("extradecraftinter","td_inter_")]:
        v = row.get(field, np.nan)
        if pd.isna(v): bucket = "unknown"
        elif float(v) <= 1/3: bucket = "L"
        elif float(v) <= 2/3: bucket = "M"
        else: bucket = "H"
        toks.append(prefix + bucket)

    return " ".join(sorted(set(toks)))

ex_docs = (
    df_exs.assign(doc=lambda d: d.apply(tokens_for_exercise, axis=1))
       [["exid","doc"]]
       .dropna(subset=["exid"])
       .astype({"exid": int})
       .set_index("exid")["doc"]
)

print("Built exercise docs:", ex_docs.shape)
pd.set_option("display.max_colwidth", None)
print(ex_docs.head(1).to_string())



Built exercise docs: (30,)
exid
1    aud_network aud_secops cat_dark_web cat_malware complexity_L group_blackmatter group_conti group_darkside group_fin6 group_lockbit group_revil group_ta505 group_unc1878 group_wizard_spider len_short maturity_H soft_trickbot struct_developments struct_exfiltration struct_impact struct_media struct_notification struct_privacy struct_response struct_validation td_inter_H td_intra_L threat_ransomware ttp_access_token_manipulation ttp_credential_dumping ttp_defense_evasion ttp_process_injection


In [28]:
# lets chceck the document we created... 
ex_docs.values

array(['aud_network aud_secops cat_dark_web cat_malware complexity_L group_blackmatter group_conti group_darkside group_fin6 group_lockbit group_revil group_ta505 group_unc1878 group_wizard_spider len_short maturity_H soft_trickbot struct_developments struct_exfiltration struct_impact struct_media struct_notification struct_privacy struct_response struct_validation td_inter_H td_intra_L threat_ransomware ttp_access_token_manipulation ttp_credential_dumping ttp_defense_evasion ttp_process_injection',
       'aud_media aud_secops cat_data_breach cat_financial complexity_H group_carbanak group_dridex_group group_evil_corp group_ursnif_operators len_medium maturity_H soft_dridex soft_trickbot struct_detection struct_developments struct_exfiltration struct_impact struct_legal struct_media struct_notification struct_privacy struct_response struct_validation td_inter_H td_intra_H threat_banking_trojan ttp_malicious_file ttp_persistence ttp_valid_accounts',
       'aud_media aud_secops cat_cry

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# We already space-join tokens; keep case as-is & treat any non-space as a token char.
tfidf = TfidfVectorizer(lowercase=False, token_pattern=r"[^ ]+")
X_ex  = tfidf.fit_transform(ex_docs.values)   # rows align to ex_docs.index order

print("DTM shape (exercises × features):", X_ex.shape)

ex_sim = cosine_similarity(X_ex, X_ex)
ex_sim_df = pd.DataFrame(ex_sim, index=ex_docs.index, columns=ex_docs.index)
print("Similarity matrix shape:", ex_sim_df.shape)



DTM shape (exercises × features): (30, 200)
Similarity matrix shape: (30, 30)


In [18]:
ex_sim_df

exid,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
exid,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.201992,0.595990,0.081390,0.203916,0.131158,0.407433,0.538270,0.497302,0.110227,...,0.198301,0.155814,0.144767,0.116216,0.111654,0.116499,0.091780,0.224072,0.149079,0.141483
2,0.201992,1.000000,0.257887,0.140975,0.241546,0.169245,0.130048,0.241232,0.111717,0.257331,...,0.167391,0.276567,0.563333,0.119260,0.198641,0.250807,0.401717,0.103075,0.292136,0.216349
3,0.595990,0.257887,1.000000,0.063856,0.256192,0.102120,0.389936,0.542230,0.532632,0.178391,...,0.129813,0.112403,0.151685,0.079067,0.150210,0.142881,0.107114,0.185478,0.204050,0.188352
4,0.081390,0.140975,0.063856,1.000000,0.103175,0.418797,0.100134,0.086025,0.127208,0.139048,...,0.054894,0.106836,0.159979,0.310947,0.113554,0.121806,0.270989,0.060225,0.204009,0.100791
5,0.203916,0.241546,0.256192,0.103175,1.000000,0.120261,0.414930,0.099144,0.174767,0.226446,...,0.093465,0.573426,0.287052,0.102832,0.136101,0.156225,0.180492,0.126002,0.185266,0.462018
6,0.131158,0.169245,0.102120,0.418797,0.120261,1.000000,0.193796,0.080041,0.078874,0.115907,...,0.195117,0.063406,0.129817,0.055966,0.111524,0.310400,0.062711,0.050598,0.132457,0.168345
7,0.407433,0.130048,0.389936,0.100134,0.414930,0.193796,1.000000,0.459123,0.454694,0.135174,...,0.140729,0.445293,0.201090,0.144048,0.085256,0.109314,0.092283,0.107781,0.227976,0.330790
8,0.538270,0.241232,0.542230,0.086025,0.099144,0.080041,0.459123,1.000000,0.567017,0.208244,...,0.057881,0.211929,0.074042,0.139519,0.037734,0.073415,0.105037,0.124959,0.085141,0.089016
9,0.497302,0.111717,0.532632,0.127208,0.174767,0.078874,0.454694,0.567017,1.000000,0.156460,...,0.044238,0.144620,0.174520,0.143268,0.075313,0.093132,0.100655,0.120154,0.129234,0.078616


In [17]:
def similar_exercises(target_exid: int, top_n: int = 5) -> pd.DataFrame:
    if target_exid not in ex_sim_df.index:
        raise KeyError(f"exid {target_exid} not found")
    s = ex_sim_df.loc[target_exid].drop(index=target_exid)
    s = s.sort_values(ascending=False).head(top_n).round(4)
    out = s.reset_index()
    out.columns = ["exid","similarity"]
    return out

print(similar_exercises(1, 5))
print(similar_exercises(10, 5))


   exid  similarity
0     3      0.5960
1     8      0.5383
2     9      0.4973
3     7      0.4074
4    18      0.2377
   exid  similarity
0    30      0.4520
1    21      0.4277
2    23      0.4268
3    18      0.4143
4    12      0.3280


In [21]:
def tokens_for_org(row):
    toks = []
    for field, prefix in [("threats","threat_"), ("ttps","ttp_"), ("aims","aim_")]:
        for val in split_semicolons(row.get(field, "")):
            toks.append(prefix + norm_token(val))
    # Bucket maturity/complexity to match exercise tokens
    mat = bucket_numeric(row.get("maturity", np.nan),   bins=[2,3,5], labels=["L","M","H"])
    cpx = bucket_numeric(row.get("complexity", np.nan), bins=[2,3,5], labels=["L","M","H"])
    toks += [f"maturity_{mat}", f"complexity_{cpx}"]
    # Optional org size
    if "size" in row and isinstance(row["size"], str) and row["size"].strip():
        toks.append("size_" + norm_token(row["size"]))
    return " ".join(sorted(set(toks)))

org_docs = (
    df_orgs.assign(doc=lambda d: d.apply(tokens_for_org, axis=1))
        [["orgid","doc"]]
        .dropna(subset=["orgid"])
        .astype({"orgid": int})
        .set_index("orgid")["doc"]
)

X_org = tfidf.transform(org_docs.values)      # same vocabulary
org_ex_sim = cosine_similarity(X_org, X_ex)
org_ex_sim_df = pd.DataFrame(org_ex_sim, index=org_docs.index, columns=ex_docs.index)

def recommend_for_org(org_id: int, top_n: int = 5) -> pd.DataFrame:
    if org_id not in org_ex_sim_df.index:
        raise KeyError(f"orgid {org_id} not found")
    s = org_ex_sim_df.loc[org_id].sort_values(ascending=False).head(top_n).round(4)
    out = s.reset_index()
    out.columns = ["exid","similarity"]
    return out

# EXAMPLE
print(recommend_for_org(1, 5))


   exid  similarity
0    27      0.2214
1    14      0.2131
2    21      0.1675
3    11      0.1348
4    19      0.1321


In [24]:
def reweight_by_prefix(X, vocab: dict, prefix: str, factor: float):
    idxs = [i for tok, i in vocab.items() if tok.startswith(prefix)]
    if not idxs: return X
    X = X.tocsc(copy=True)
    X[:, idxs] = X[:, idxs] * factor
    return X.tocsr()

# Rebuild a weighted similarity using TTP weight = 3.0
vocab = tfidf.vocabulary_
X_ex_w = reweight_by_prefix(X_ex, vocab, "ttp_", 2.0)
ex_sim_w = cosine_similarity(X_ex_w, X_ex_w)
ex_sim_w_df = pd.DataFrame(ex_sim_w, index=ex_docs.index, columns=ex_docs.index)

def most_similar_exercises_weighted(target_exid: int, top_n: int = 5) -> pd.DataFrame:
    if target_exid not in ex_sim_w_df.index:
        raise KeyError(f"exid {target_exid} not found")
    s = ex_sim_w_df.loc[target_exid].drop(index=target_exid)
    s = s.sort_values(ascending=False).head(top_n).round(4)
    out = s.reset_index()
    out.columns = ["exid","similarity"]
    return out

# EXAMPLE
print(most_similar_exercises_weighted(1, 5))


   exid  similarity
0     3      0.3642
1     8      0.3346
2     9      0.3300
3     5      0.2486
4     7      0.2431


In [26]:
org_ex_sim_df

exid,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
orgid,,,,,,,,,,,,,,,,,,,,,
1,0.038761,0.086203,0.032469,0.114165,0.035950,0.059440,0.058320,0.034995,0.120848,0.037899,...,0.167459,0.036202,0.093666,0.086886,0.063671,0.000000,0.221417,0.129734,0.085689,0.000000
2,0.056434,0.028951,0.061061,0.061485,0.067609,0.201508,0.153326,0.065812,0.041713,0.223145,...,0.148900,0.026243,0.031457,0.062983,0.084036,0.211112,0.069374,0.044780,0.113096,0.236126
3,0.129130,0.167021,0.061658,0.181506,0.119395,0.080595,0.110152,0.123074,0.051100,0.018313,...,0.080918,0.144258,0.099740,0.000000,0.114481,0.092649,0.096562,0.072985,0.116186,0.000000
4,0.000000,0.102247,0.059937,0.041938,0.125880,0.019595,0.019226,0.064599,0.126933,0.069961,...,0.000000,0.025759,0.182941,0.086207,0.020990,0.136907,0.216850,0.136266,0.028248,0.033531
5,0.029086,0.064685,0.024364,0.022020,0.026976,0.044603,0.111649,0.114183,0.000000,0.028439,...,0.032300,0.118123,0.070285,0.147965,0.112633,0.000000,0.084837,0.000000,0.064299,0.151444
6,0.126617,0.000000,0.000000,0.000000,0.141316,0.000000,0.123812,0.000000,0.000000,0.000000,...,0.046766,0.116446,0.000000,0.046999,0.053428,0.099383,0.143498,0.048765,0.071903,0.078739
7,0.039628,0.052816,0.026350,0.116351,0.071752,0.119772,0.161370,0.028400,0.090805,0.193924,...,0.203982,0.000000,0.116114,0.082928,0.208370,0.137881,0.083102,0.301489,0.059208,0.181252
8,0.106411,0.000000,0.035378,0.000000,0.039172,0.000000,0.000000,0.038131,0.039327,0.041296,...,0.059085,0.000000,0.000000,0.186956,0.094125,0.126023,0.045745,0.227463,0.000000,0.079206
9,0.067562,0.026805,0.110480,0.115286,0.000000,0.018483,0.155941,0.000000,0.000000,0.064336,...,0.138105,0.000000,0.029125,0.100679,0.069874,0.163701,0.055808,0.052231,0.094037,0.090019
